# Evaluate loglikelihood

__Note__: Run after experiments_01_benchmark_synthetic_survival_data.ipynb

In [ ]:
!pip install autoprognosis

In [1]:
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import synthcity.logger as log
from sklearn.mixture import GaussianMixture as GMM

from datasets import get_dataset

warnings.filterwarnings("ignore", category=FutureWarning)

log.remove()
warnings.filterwarnings("ignore")
log.add(sink=sys.stderr, level="DEBUG")

In [6]:
from synthcity.plugins.core.models.time_to_event import (
    get_model_template as get_tte_model_template,
)


class TabularGMM:
    def __init__(self, components: int = 100, random_state: int = 0):
        self.model = GMM(100, covariance_type="full", random_state=random_state)
        self.tte_regressor = get_tte_model_template("survival_function_regression")()

    def fit(self, X, T, E):
        self.model.fit(X)
        self.tte_regressor.fit(X, T, E)

        self.E = E
        self.count = len(X)
        self.columns = X.columns

    def generate(self, count: int = None):
        if count is None:
            count = self.count
        sampled, _ = self.model.sample(count)
        sampled = pd.DataFrame(sampled, columns=self.columns)

        E = self.E.reset_index(drop=True).head(count)

        T = pd.Series(self.tte_regressor.predict_any(sampled, E,))
        return sampled, T, E

In [7]:
import numpy as np
import scipy
from scipy.stats import norm


def censored_nll(params, durations, events):
    mu, std_dev = params
    observed_durations = durations[events == 1]  # or events==True
    censored_durations = durations[~(events == 1)]  # or events==True
    return -(
        np.log(1e-8 + norm.pdf(observed_durations, loc=mu, scale=std_dev)).sum()
        + np.log(1e-8 + norm.sf(censored_durations, loc=mu, scale=std_dev)).sum()
    )


def negative_log_likelihood(T, E):
    return scipy.optimize.minimize(
        censored_nll, method="Nelder-Mead", x0=np.array([0, 1]), args=(T, E)
    ).fun

In [13]:
import platform
from pathlib import Path

import tabulate
from autoprognosis.plugins.prediction.risk_estimation import RiskEstimation
from autoprognosis.utils.metrics import generate_score, print_score
from synthcity.plugins.core.dataloader import SurvivalAnalysisDataLoader
from synthcity.utils.serialization import dataframe_hash, load_from_file, save_to_file

log.remove()

out_dir = Path("workspace")
headers = [
    "dataset",
    "method",
    "degrees of freedom/covariates",
    "with outcome",
    "censored",
    "log_likelihood",
    "log_likelihood_ratio_test",
]

for ref_df in ["aids"]:
    distances = []

    print("=======================")
    print("Evaluate ", ref_df)

    df, duration_col, event_col, time_horizons = get_dataset(ref_df)
    df_hash = dataframe_hash(df)
    model = RiskEstimation().get("cox_ph")

    X = df.drop(columns=[duration_col, event_col])
    T = df[duration_col]
    E = df[event_col]

    model.fit(X, T, E)

    ref_log_likelihood = -model.model.model.log_likelihood_
    ref_log_likelihood_ratio_test = (
        model.model.model.log_likelihood_ratio_test().test_statistic
    )
    censored = (E == 0).sum()
    outcome = (E == 1).sum()

    distances.append(
        [
            ref_df,
            "Real data",
            X.shape[1],
            censored,
            outcome,
            ref_log_likelihood,
            ref_log_likelihood_ratio_test,
        ]
    )

    for method in ["survival_gan", "ctgan", "nflow", "tvae", "privbayes"]:
        local_log_likelihood = []
        local_log_likelihood_ratio = []
        local_censored = []
        local_outcome = []
        for seed in range(3):
            model_bkp = (
                out_dir
                / f"{df_hash}_{method}_{method}__{platform.python_version()}_{seed}.bkp"
            )
            if not model_bkp.exists():
                continue
            syn_df = load_from_file(model_bkp).dataframe()

            Xsyn = syn_df.drop(columns=[duration_col, event_col])
            Tsyn = syn_df[duration_col]
            Esyn = syn_df[event_col]
            model = RiskEstimation().get("cox_ph")
            model.fit(Xsyn, Tsyn, Esyn)

            log_likelihood = -model.model.model.log_likelihood_
            log_likelihood_ratio_test = (
                model.model.model.log_likelihood_ratio_test().test_statistic
            )
            # other_log_likelihood = negative_log_likelihood(syn_df[duration_col], syn_df[event_col])

            local_log_likelihood.append(log_likelihood)
            local_log_likelihood_ratio.append(log_likelihood_ratio_test)
            local_censored.append((Esyn == 0).sum())
            local_outcome.append((Esyn == 1).sum())

        log_likelihood_str = print_score(generate_score(local_log_likelihood))
        log_likelihood_ratio_test_str = print_score(
            generate_score(local_log_likelihood_ratio)
        )
        censored_str = print_score(generate_score(local_censored))
        outcome_str = print_score(generate_score(local_outcome))

        distances.append(
            [
                ref_df,
                method,
                X.shape[1],
                censored_str,
                outcome_str,
                log_likelihood_str,
                log_likelihood_ratio_test_str,
            ]
        )
    tabulate.tabulate(distances, headers=headers, tablefmt="html")

distances

Evaluate  aids


[['aids', 'Real data', 11, 1055, 96, 621.926143124486, 73.03276504216637],
 ['aids',
  'survival_gan',
  11,
  '1054.0 +/- 0.0',
  '97.0 +/- 0.0',
  '576.534 +/- 0.0',
  '198.483 +/- 0.0'],
 ['aids',
  'ctgan',
  11,
  'nan +/- nan',
  'nan +/- nan',
  'nan +/- nan',
  'nan +/- nan'],
 ['aids',
  'nflow',
  11,
  'nan +/- nan',
  'nan +/- nan',
  'nan +/- nan',
  'nan +/- nan'],
 ['aids',
  'tvae',
  11,
  '1092.0 +/- 0.0',
  '59.0 +/- 0.0',
  '378.374 +/- 0.0',
  '18.482 +/- 0.0'],
 ['aids',
  'privbayes',
  11,
  'nan +/- nan',
  'nan +/- nan',
  'nan +/- nan',
  'nan +/- nan']]

In [14]:
import tabulate

tabulate.tabulate(distances, headers=headers, tablefmt="html")

dataset,method,degrees of freedom/covariates,with outcome,censored,log_likelihood,log_likelihood_ratio_test
aids,Real data,11,1055,96,621.926143124486,73.03276504216637
aids,survival_gan,11,1054.0 +/- 0.0,97.0 +/- 0.0,576.534 +/- 0.0,198.483 +/- 0.0
aids,ctgan,11,nan +/- nan,nan +/- nan,nan +/- nan,nan +/- nan
aids,nflow,11,nan +/- nan,nan +/- nan,nan +/- nan,nan +/- nan
aids,tvae,11,1092.0 +/- 0.0,59.0 +/- 0.0,378.374 +/- 0.0,18.482 +/- 0.0
aids,privbayes,11,nan +/- nan,nan +/- nan,nan +/- nan,nan +/- nan
